In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data.csv")
df.head()

/var/folders/0_/5hrgkx011w96l3gk3vpn_34h0000gq/T/ipykernel_10394/2036481670.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data.csv")


,query,templatized_query,lob
0,What happens if my bus is cancelled by the ope...,What happens if my bus is cancelled by the [OP...,BUS
1,Please confirm the change to board from Bandra...,Please confirm the change to board from [SOURC...,CABS
2,What is TripAssured?,What is [OPERATOR]?,BUS
3,Check train availability from Mumbai to Bhadoh...,Check train availability from [SOURCE] to [DES...,RAILS
4,Assist in changing the boarding point for trai...,Assist in changing the boarding point for trai...,CABS


In [4]:
df['query'].duplicated().sum()

np.int64(77121)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 532503 entries, 0 to 532502
Data columns (total 3 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   query              532503 non-null  object
 1   templatized_query  99870 non-null   object
 2   lob                532503 non-null  object
dtypes: object(3)
memory usage: 12.2+ MB


In [6]:
df.isnull().sum()

query                     0
templatized_query    432633
lob                       0
dtype: int64

In [7]:
df.drop(columns=["templatized_query", "lob"], inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 532503 entries, 0 to 532502
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   query   532503 non-null  object
dtypes: object(1)
memory usage: 4.1+ MB


In [10]:
df.columns

Index(['query'], dtype='object')

In [11]:
new_df = df.drop_duplicates(subset=["query"])

In [12]:
new_df.head()

,query
0,What happens if my bus is cancelled by the ope...
1,Please confirm the change to board from Bandra...
2,What is TripAssured?
3,Check train availability from Mumbai to Bhadoh...
4,Assist in changing the boarding point for trai...


In [13]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 455382 entries, 0 to 532502
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   query   455382 non-null  object
dtypes: object(1)
memory usage: 6.9+ MB


In [14]:
new_df.duplicated().sum()

np.int64(0)

In [15]:
new_df.to_csv("filtered_queries.csv", index=False)

In [16]:
def is_mostly_ascii(text):
    try:
        # Allow some non-ASCII chars like ₹, but reject if more than 5% are non-ASCII letters
        text = str(text)
        non_ascii_letters = sum(1 for c in text if ord(c) > 127 and c.isalpha())
        total_letters = sum(1 for c in text if c.isalpha())
        if total_letters == 0:
            return True
        return (non_ascii_letters / total_letters) < 0.05
    except:
        return False

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 532503 entries, 0 to 532502
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   query   532503 non-null  object
dtypes: object(1)
memory usage: 4.1+ MB


In [18]:
df_english = df[df['query'].apply(is_mostly_ascii)]


In [19]:
df_english.info()

<class 'pandas.core.frame.DataFrame'>
Index: 531432 entries, 0 to 532502
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   query   531432 non-null  object
dtypes: object(1)
memory usage: 8.1+ MB


In [20]:
df_english.to_csv('filtered_queries_english_only.csv', index=False)

In [21]:
df_eng_filtered = pd.read_csv('filtered_queries_english_only.csv')
df_eng_filtered.head()



,query
0,What happens if my bus is cancelled by the ope...
1,Please confirm the change to board from Bandra...
2,What is TripAssured?
3,Check train availability from Mumbai to Bhadoh...
4,Assist in changing the boarding point for trai...


In [22]:
def has_multiple_words(text):
    words = str(text).strip().split()
    return len(words) > 1

In [23]:
df_eng_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 531432 entries, 0 to 531431
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   query   531432 non-null  object
dtypes: object(1)
memory usage: 4.1+ MB


In [24]:
df_eng_filtered = df[df['query'].apply(has_multiple_words)]

In [25]:
df_eng_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 530487 entries, 0 to 532502
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   query   530487 non-null  object
dtypes: object(1)
memory usage: 8.1+ MB


In [26]:
df_eng_filtered.to_csv('filtered_queries_multiple_words.csv', index=False)

In [27]:
import pandas as pd
import json

# Read the CSV
df = pd.read_csv('filtered_queries_english_only.csv')

# Convert to simple list of strings
queries = df['query'].tolist()

# Save as JSON
with open('filtered_queries_english_only.json', 'w', encoding='utf-8') as f:
    json.dump(queries, f, indent=2, ensure_ascii=False)

print(f"✅ Converted {len(queries)} queries to JSON list")
print("Output: filtered_queries_english_only.json")

✅ Converted 531432 queries to JSON list
Output: filtered_queries_english_only.json


In [28]:
import json

# Read the JSON file
with open('filtered_queries_english_only.json', 'r', encoding='utf-8') as f:
    queries = json.load(f)

print(f"Total queries before removing duplicates: {len(queries)}")

# Remove exact duplicates while preserving order
unique_queries = list(dict.fromkeys(queries))

print(f"Total unique queries: {len(unique_queries)}")
print(f"Duplicates removed: {len(queries) - len(unique_queries)}")

# Save back to JSON
with open('filtered_queries_english_only.json', 'w', encoding='utf-8') as f:
    json.dump(unique_queries, f, indent=2, ensure_ascii=False)

print("✅ Duplicates removed and file updated!")


Total queries before removing duplicates: 531432
Total unique queries: 454330
Duplicates removed: 77102
✅ Duplicates removed and file updated!


In [29]:
import json

# Read the JSON file
with open('filtered_queries_english_only.json', 'r', encoding='utf-8') as f:
    queries = json.load(f)

print(f"Total queries before cleaning: {len(queries)}")

# Remove null, empty, and whitespace-only queries
cleaned_queries = [
    query for query in queries 
    if query and isinstance(query, str) and query.strip()
]

print(f"Total valid queries: {len(cleaned_queries)}")
print(f"Null/empty queries removed: {len(queries) - len(cleaned_queries)}")

# Save back to JSON
with open('filtered_queries_english_only.json', 'w', encoding='utf-8') as f:
    json.dump(cleaned_queries, f, indent=2, ensure_ascii=False)

print("✅ Null/empty queries removed and file updated!")

Total queries before cleaning: 454330
Total valid queries: 454330
Null/empty queries removed: 0
✅ Null/empty queries removed and file updated!


In [30]:
import json

# Read the JSON file
with open('filtered_queries_english_only.json', 'r', encoding='utf-8') as f:
    queries = json.load(f)

print(f"Total queries: {len(queries)}")

# Sort in ascending order (alphabetically, case-insensitive)
sorted_queries = sorted(queries, key=str.lower)

print("✅ Queries sorted in ascending order")

# Save to a NEW file
with open('filtered_queries_sorted.json', 'w', encoding='utf-8') as f:
    json.dump(sorted_queries, f, indent=2, ensure_ascii=False)

print("✅ Sorted queries saved to: filtered_queries_sorted.json")
print(f"Original file (filtered_queries_english_only.json) remains unchanged")

Total queries: 454330
✅ Queries sorted in ascending order
✅ Sorted queries saved to: filtered_queries_sorted.json
Original file (filtered_queries_english_only.json) remains unchanged


In [31]:
import json

def should_keep_query(query):
    """
    Keep query if:
    - It doesn't contain 'cab' or 'train', OR
    - It contains 'cab' or 'train' BUT also contains 'bus'
    
    Remove query if:
    - It contains 'cab' or 'train' AND doesn't contain 'bus'
    """
    query_lower = query.lower()
    
    has_cab = 'cab' in query_lower
    has_train = 'train' in query_lower
    has_bus = 'bus' in query_lower
    
    # If it has cab or train, only keep if it also has bus
    if has_cab or has_train:
        return has_bus
    
    # If it doesn't have cab or train, keep it
    return True


def filter_queries(input_file, output_file):
    """Filter queries and save to output file"""
    
    print(f"Reading from: {input_file}")
    with open(input_file, 'r', encoding='utf-8') as f:
        queries = json.load(f)
    
    print(f"Total queries before filtering: {len(queries)}")
    
    # Filter queries
    filtered_queries = [q for q in queries if should_keep_query(q)]
    
    print(f"Total queries after filtering: {len(filtered_queries)}")
    print(f"Removed: {len(queries) - len(filtered_queries)} queries")
    
    # Save filtered queries
    print(f"Writing to: {output_file}")
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(filtered_queries, f, ensure_ascii=False, indent=2)
    
    print("✓ Filtering complete!")
    
    # Show some examples of what was removed
    removed = [q for q in queries if not should_keep_query(q)]
    if removed:
        print("\nExample of removed queries (first 10):")
        for q in removed[:10]:
            print(f"  - {q}")


if __name__ == "__main__":
    input_file = "/Users/int1964/TEMPLATE_GENRATOR/filtered_queries_sorted.json"
    output_file = "/Users/int1964/TEMPLATE_GENRATOR/filtered_queries_bus_only.json"
    
    filter_queries(input_file, output_file)

Reading from: /Users/int1964/TEMPLATE_GENRATOR/filtered_queries_sorted.json
Total queries before filtering: 453872
Total queries after filtering: 211952
Removed: 241920 queries
Writing to: /Users/int1964/TEMPLATE_GENRATOR/filtered_queries_bus_only.json
✓ Filtering complete!

Example of removed queries (first 10):
  - A 4-seater cab can comfortably accommodate up to 4 people.
  - A new train ticket has become available. Can you provide me with the details about this ticket?
  - A TT is coming down at the railway station in Mumbai. Can you provide information about this train and its schedule for my journey from Mumbai to Lucknow?
  - Aaj 2026 ko Mau se Jaipur ke liye koi seedhi train nahi mili. Ab jab aapke paas apratyaksh train vikalp hai, kya aapko train ke prakaar jaise sleeper ya AC ke liye koi prathmikta hai ya kya aap samay saarini dekhna chahenge?
  - Aaj untees tarikh ko Lucknow se Bhopal jaane Wali kaun si train thi shaam ko paanch baje?
  - Aapko train yatra mein '1A', '2A', a

In [32]:
import json

# Define filter keyword lists for NON-BUS transport
FLIGHT_KEYWORDS = ['flight', 'airline', 'airways', 'aircraft', 'airport', 'terminal', 
                   'boarding pass', 'check-in counter']
TRAIN_KEYWORDS = ['train', 'irctc', 'rail', 'tatkal', 'pnr', 'rac', 'waitlist', 'waitlisted',
                  'chart preparation', 'railway', 'berth', '1a', '2a', '3a', 
                  'rajdhani', 'shatabdi', 'express train', 'duronto', 'coach number']
HOTEL_KEYWORDS = ['hotel', 'resort', 'stay at', 'accommodation', 'room booking',
                  'check-in time', 'check-out', 'nights stay', 'lodge', 'guest house']
CAR_KEYWORDS = ['cab', 'taxi', 'car rental', 'outstation car', 'sedan', 'suv',
                'hatchback', 'tempo traveller', 'innova', 'ertiga', 
                'roof carrier', 'per km', 'km charges', 'driver details',
                'driver language', 'luggage capacity', 'toll tax']

def should_remove_query(query):
    """
    Returns True if query should be REMOVED (non-bus transport), False if should be kept
    
    REMOVE only if:
    - Contains flight/train/hotel/car keywords AND doesn't also mention "bus"
    
    KEEP if:
    - Contains "bus" (even if also mentions train/cab/etc)
    - Doesn't contain any of the filter keywords (neutral/generic query)
    """
    query_lower = query.lower()
    
    # Check if query mentions "bus" - if yes, always KEEP
    if 'bus' in query_lower or 'buses' in query_lower:
        return False  # Don't remove
    
    # Check if too short (likely just noise)
    if len(query.strip()) < 5:
        return True  # Remove
    
    # Check for explicit non-bus transport keywords
    has_flight = any(keyword in query_lower for keyword in FLIGHT_KEYWORDS)
    has_train = any(keyword in query_lower for keyword in TRAIN_KEYWORDS)
    has_hotel = any(keyword in query_lower for keyword in HOTEL_KEYWORDS)
    has_car = any(keyword in query_lower for keyword in CAR_KEYWORDS)
    
    # REMOVE only if it explicitly mentions other transport modes
    if has_flight or has_train or has_hotel or has_car:
        return True  # Remove - clearly about other transport
    
    # Otherwise KEEP (neutral/generic query that might be relevant)
    return False  # Don't remove


def filter_queries(input_file, output_file):
    """Filter queries - remove only explicit non-bus queries"""
    
    print(f"📖 Reading from: {input_file}")
    with open(input_file, 'r', encoding='utf-8') as f:
        queries = json.load(f)
    
    print(f"📊 Total queries before filtering: {len(queries):,}")
    
    # Filter queries - keep those that should NOT be removed
    filtered_queries = [q for q in queries if not should_remove_query(q)]
    
    print(f"✅ Total queries after filtering: {len(filtered_queries):,}")
    print(f"❌ Removed: {len(queries) - len(filtered_queries):,} queries")
    print(f"📈 Retention rate: {len(filtered_queries)/len(queries)*100:.2f}%")
    
    # Save filtered queries
    print(f"\n💾 Writing to: {output_file}")
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(filtered_queries, f, ensure_ascii=False, indent=2)
    
    print("✅ Filtering complete!")
    
    # Show examples of what was removed
    removed = [q for q in queries if should_remove_query(q)]
    if removed:
        print("\n🗑️  Examples of REMOVED queries (first 15):")
        for i, q in enumerate(removed[:15], 1):
            print(f"  {i}. {q[:100]}{'...' if len(q) > 100 else ''}")
    
    # Show examples of what was kept (non-bus but generic)
    kept_neutral = [q for q in filtered_queries 
                   if 'bus' not in q.lower() and len(q) > 10][:15]
    if kept_neutral:
        print("\n✅ Examples of KEPT queries (neutral/generic, no 'bus' mention):")
        for i, q in enumerate(kept_neutral[:15], 1):
            print(f"  {i}. {q[:100]}{'...' if len(q) > 100 else ''}")


if __name__ == "__main__":
    input_file = "/Users/int1964/TEMPLATE_GENRATOR/filtered_queries_bus_only.json"
    output_file = "/Users/int1964/TEMPLATE_GENRATOR/filtered_queries_final.json"
    
    filter_queries(input_file, output_file)

📖 Reading from: /Users/int1964/TEMPLATE_GENRATOR/filtered_queries_bus_only.json
📊 Total queries before filtering: 210,245
✅ Total queries after filtering: 165,650
❌ Removed: 44,595 queries
📈 Retention rate: 78.79%

💾 Writing to: /Users/int1964/TEMPLATE_GENRATOR/filtered_queries_final.json
✅ Filtering complete!

🗑️  Examples of REMOVED queries (first 15):
  1. A 1A, 2A, 3A ka matlab Hindi mein samjhaen.
  2. A makes a booking of 2 tickets from Delhi to Jaipur through the MMT platform under the Alternate Tri...
  3. Aadhaar is not connected. Can you please help me with that? I'm unable to connect with IRCTC and I t...
  4. Aami ekti notun irctc account banaate chaychhi.
  5. About cancellation as if flight is late then?
  6. About pnr is confirmed?
  7. above reference  for Lucknow to Bangalore  by Akasa Air  flight cancelled by airline so i need my fu...
  8. ac 
  9. Accommodation for driver?
  10. According to IRCTC rules, how much refund will I receive for a waitlisted ticket if I do

In [33]:
import csv

input_file = 'data.csv'
temp_file = 'data_temp.csv'

removed_count = 0
kept_count = 0

# Write to temp file
with open(input_file, 'r', encoding='utf-8') as infile, \
     open(temp_file, 'w', encoding='utf-8', newline='') as outfile:
    
    reader = csv.reader(infile)
    writer = csv.writer(outfile)
    
    # Write header
    header = next(reader)
    writer.writerow(header)
    
    # Process each row
    for row in reader:
        if len(row) >= 2 and row[1].strip():
            writer.writerow(row)
            kept_count += 1
        else:
            removed_count += 1

# Replace original with cleaned version
import os
os.replace(temp_file, input_file)

print(f"✅ File cleaned: {input_file}")
print(f"📊 Kept: {kept_count:,} rows")
print(f"🗑️  Removed: {removed_count:,} rows with null/empty entities")

✅ File cleaned: data.csv
📊 Kept: 99,870 rows
🗑️  Removed: 432,633 rows with null/empty entities


In [34]:
import csv

input_file = 'originalData.csv'
output_file = 'originalData_cleaned.csv'

removed_count = 0
kept_count = 0

with open(input_file, 'r', encoding='utf-8') as infile, \
     open(output_file, 'w', encoding='utf-8', newline='') as outfile:
    
    reader = csv.reader(infile)
    writer = csv.writer(outfile)
    
    # Read header and drop the 'templatized_query' column (index 1)
    header = next(reader)
    # Write only 'query' and 'lob' columns (indices 0 and 2)
    writer.writerow([header[0], header[2]])  # query, lob
    
    # Process each row
    for row in reader:
        # Check if query column (index 0) is not empty
        if len(row) >= 1 and row[0].strip():
            # Write only query and lob columns (drop templatized_query)
            writer.writerow([row[0], row[2] if len(row) > 2 else ''])
            kept_count += 1
        else:
            removed_count += 1

print(f"✅ Cleaned CSV created: {output_file}")
print(f"📊 Kept: {kept_count:,} rows")
print(f"🗑️  Removed: {removed_count:,} rows with null/empty queries")
print(f"🔧 Dropped column: 'templatized_query'")
print(f"📝 Columns remaining: 'query', 'lob'")

✅ Cleaned CSV created: originalData_cleaned.csv
📊 Kept: 532,503 rows
🗑️  Removed: 0 rows with null/empty queries
🔧 Dropped column: 'templatized_query'
📝 Columns remaining: 'query', 'lob'


In [35]:
import pandas as pd

input_file = 'originalData.csv'
output_file = 'originalData_bus_only.csv'

# Read CSV
df = pd.read_csv(input_file)

print(f"📊 Total rows: {len(df):,}")
print(f"\n🏷️  LOB distribution:")
print(df['lob'].value_counts())

# Keep only BUS rows
df_bus = df[df['lob'] == 'BUS']

# Save to new file
df_bus.to_csv(output_file, index=False)

print(f"\n✅ BUS-only CSV created: {output_file}")
print(f"📊 BUS rows kept: {len(df_bus):,}")
print(f"🗑️  Non-BUS rows removed: {len(df) - len(df_bus):,}")

/var/folders/0_/5hrgkx011w96l3gk3vpn_34h0000gq/T/ipykernel_10394/2774160550.py:7: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_file)


📊 Total rows: 532,503

🏷️  LOB distribution:
lob
COMMONS              197815
RAILS                128119
CABS                  90713
BUS                   65654
COMMON                26499
FLIGHTS_POSTSALES      8463
WHERE2GO               7381
HOTELS                 4255
FLIGHTS                2151
HOLIDAYS                459
TRAINS                  367
UNKNOWN                 298
FOREX                   227
DESTINATION              31
CABS_POST_SALES          19
INSURANCE                18
VISA                     14
POSTSALES_CAB            10
CAB                      10
Name: count, dtype: int64

✅ BUS-only CSV created: originalData_bus_only.csv
📊 BUS rows kept: 65,654
🗑️  Non-BUS rows removed: 466,849
